In [68]:
from dotenv import load_dotenv
load_dotenv()
from google.cloud.firestore import Client as FirestoreClient
from chat_history_service import ChatHistoryService
from llm_request_service import LLMRequestService
from typing import List, Dict, Any

## Define Functions

In [89]:
def get_chat_history_from_firestore(session_id: str) -> list[dict]:
    """
    Fetch chat history from Firestore for a given session ID.
    
    Args:
        session_id (str): The session ID to fetch chat history for.
    
    Returns:
        list[dict]: List of chat messages in the session.
    """
    chat_history_service = ChatHistoryService()
    chat_history = chat_history_service.get_chat_history_by_id(session_id)
    chat_history = sorted(chat_history, key=lambda x: x.get("create_time", ""))
    print(chat_history)
    return chat_history

In [107]:
def get_specific_data_from_history(history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Retrieves specific data from the entire conversation history, maintaining the conversation flow.
    
    Args:
        history (List[Dict[str, Any]]): List of dictionaries containing message history.
    
    Returns:
        List[Dict[str, Any]]: List of dictionaries, each containing user message, bot responses, and button label for each turn in the conversation.
    """
    # Sort the history by create_time
    sorted_history = sorted(history, key=lambda x: x.get("create_time"))
    
    # Initialize list to store the conversation turns
    conversation = []
    
    # Iterate through the history
    for message in sorted_history:
        turn = {}
        
        # Get user message
        user_msg = message.get("user_msg")
        if user_msg:
            turn["user_msg"] = user_msg
        
        # Get bot responses
        bot_response = message.get("bot_response", [])
        if isinstance(bot_response, list):
            bot_messages = []
            for response in bot_response:
                if isinstance(response, dict) and "message" in response:
                    bot_messages.append({"message": response["message"]})
            if bot_messages:
                turn["bot_response"] = bot_messages
        
        # Get button label
        button_label = message.get("button_label")
        if button_label:
            turn["button_label"] = button_label
        
        # Add the turn to the conversation if it's not empty
        if turn:
            conversation.append(turn)
    
    return conversation

In [94]:
def get_specific_chat_data(chat_history: str):
    """
    Fetches chat history from Firestore and selects specific data.

    Args:
        session_id (str): The session ID to fetch chat history for.

    Returns:
        Dict[str, Any]: Selected specific data from the chat history.
    """
    specific_chat_history = get_specific_data_from_history(chat_history)
    return specific_chat_history

In [111]:
def call_llm_service_summarize(conversation: str, prompt: str = "") -> str:
    """
    Call the LLM service to summarize the chat history.
    Args:
        conversation (str): The chat history to summarize.
        prompt (str): The prompt to send to the LLM service.
            If not provided, the default prompt will be fetched from Firestore.
    Returns:
        str: The summary of the chat history.
    """
    llm_request_service_summarize = LLMRequestService()
    chat_history_summary = llm_request_service_summarize.summarize_conversation(
        conversation, prompt
    )
    return chat_history_summary

In [112]:
def format_summary(summary: str) -> str:
    # Split the summary into sentences
    sentences = summary.split('. ')
    # Join the sentences with line breaks
    formatted_summary = '.\n'.join(sentences)
    return f"Summary:\n{formatted_summary}"

In [124]:
def count_output(text):
    """
    Count characters, words, and lines in the given text.
    """
    char_count = len(text)
    word_count = len(text.split())
    return char_count, word_count

In [113]:
def call_llm_service_tag(summary) -> str:
    """
    Call the LLM service to summarize the chat history.
    Args:
        conversation (str): The chat history to summarize.
        prompt (str): The prompt to send to the LLM service.
            If not provided, the default prompt will be fetched from Firestore.
    Returns:
        str: The summary of the chat history.
    """
    llm_request_service_tag = LLMRequestService()
    tag_queue = llm_request_service_tag.tag_callback(
        summary
    )
    return tag_queue

## Gather Conversation

### Get conversation from session_id in Development

In [114]:
session_id = "20e62f9c-be3e-4d6e-978b-7d67dd5f10e8" # Replace with your actual session ID
chat_history = get_chat_history_from_firestore(session_id)
chat_history

[{'response_time': DatetimeWithNanoseconds(2025, 7, 24, 8, 24, 35, 205532, tzinfo=datetime.timezone.utc), 'user_msg': None, 'button_label': '', 'sender': 'bot', 'bot_response': [{'failed_event_trigger': None, 'billing_account': None, 'welcome_short': None, 'msisdnflag': None, 'html_en': None, 'escapeOption': None, 'html': None, 'user_type': None, 'platform': None, 'language': None, 'randomizer': None, 'fallback_length': None, 'product': None, 'attachment': None, 'bullets_en': None, 'numbers_en': None, 'type': 'text', 'buttons': None, 'subscription_brand': None, 'video': None, 'message_en': None, 'linkList': None, 'link': None, 'numbers': None, 'message': 'Goedemorgen, ik ben chatbot Izzi van Odido.', 'image': None, 'bullets': None}, {'failed_event_trigger': None, 'billing_account': None, 'welcome_short': None, 'msisdnflag': None, 'html_en': None, 'escapeOption': False, 'html': None, 'user_type': None, 'platform': None, 'language': 'nl', 'randomizer': None, 'fallback_length': None, 'pro

[{'response_time': DatetimeWithNanoseconds(2025, 7, 24, 8, 24, 35, 205532, tzinfo=datetime.timezone.utc),
  'user_msg': None,
  'button_label': '',
  'sender': 'bot',
  'bot_response': [{'failed_event_trigger': None,
    'billing_account': None,
    'welcome_short': None,
    'msisdnflag': None,
    'html_en': None,
    'escapeOption': None,
    'html': None,
    'user_type': None,
    'platform': None,
    'language': None,
    'randomizer': None,
    'fallback_length': None,
    'product': None,
    'attachment': None,
    'bullets_en': None,
    'numbers_en': None,
    'type': 'text',
    'buttons': None,
    'subscription_brand': None,
    'video': None,
    'message_en': None,
    'linkList': None,
    'link': None,
    'numbers': None,
    'message': 'Goedemorgen, ik ben chatbot Izzi van Odido.',
    'image': None,
    'bullets': None},
   {'failed_event_trigger': None,
    'billing_account': None,
    'welcome_short': None,
    'msisdnflag': None,
    'html_en': None,
    'escap

In [115]:
selected_chat_data = get_specific_chat_data(chat_history)
selected_chat_data

[{'bot_response': [{'message': 'Goedemorgen, ik ben chatbot Izzi van Odido.'},
   {'message': 'Over welk product wil je iets vragen?'}]},
 {'bot_response': [{'message': 'Komen we er samen niet uit, dan zijn onze adviseurs er voor je.'},
   {'message': 'Wat kan ik voor je doen?'}],
  'button_label': 'Mobiel'},
 {'user_msg': 'videoland',
  'bot_response': [{'message': 'Met Videoland kijk je de beste series uit binnen- en buitenland. Het aanbod is erg ruim. Zo zit er voor iedereen wat tussen.'},
   {'message': 'Kijk Videoland waar en wanneer je maar wil. Thuis op de bank, of onderweg in de trein. Erg handig.'},
   {'message': 'Waarmee kan ik je helpen?'}]},
 {'bot_response': [{'message': 'Je wil films en series streamen bij Videoland. Dat snap ik.'},
   {'message': 'Je kan een abonnement aanvragen via de Videoland website. Of in Mijn Odido. Dan heb je alles op 1 plek.'},
   {'message': 'Hoe wil je Videoland aanvragen?'}],
  'button_label': 'Aanvragen'}]

## Manage Prompt

In [128]:
# Create a prompt that eventually contains the variable {text}
# The chat history will be concatenated into this prompt via this variable
prompt = """
Make a summary of this chat {text}. Describe why the customer is contacting Odido. The customers input is shown in "user_msg" and "button_label". Exclude any greetings or farewells. 
The summary should be concise and to the point. Max 100 characters. The output should be in Dutch.
"""

## Generate Summary

In [129]:
summary = call_llm_service_summarize(conversation, prompt)
formatted_output = format_summary(summary)
print(formatted_output)
print()

# Count the output
char_count, word_count = count_output(formatted_output)
print(f"Characters: {char_count}")
print(f"Words: {word_count}")

Summary:
De klant vraagt naar informatie over Videoland en hoe een abonnement aan te vragen.

Characters: 92
Words: 15


## Update the prompt in Firestore Database (code uses the uploaded one)

In [ ]:
# firestore_client = FirestoreClient()
# collection = firestore_client.collection("llm-prompts")
# document = collection.document("fulfillment-webhook")
# document.update({"summarize-chat-conversation": prompt})